In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib import cm
from io import StringIO
import pandas as pd

In [2]:
data = fits.open('/Users/praewapon/Desktop/Snr Proj/manga-8135-1902-LOGCUBE.fits')

In [3]:
flux = np.transpose(data['FLUX'].data, axes=(2, 1, 0))
ivar = np.transpose(data['IVAR'].data, axes=(2, 1, 0))
mask = np.transpose(data['MASK'].data, axes=(2, 1, 0))
wave = data['WAVE'].data

In [4]:
mangaid = '1-43214'
redshift = 0.117997
plateifu = '8135-1902'

In [5]:
do_not_use = (mask & 2**10) != 0 

In [8]:
OIII_5007 = []

wave_5007 = 5007 * (1 + redshift)
upper_b = 5630
lower_b = 5580

for x in range (len(flux)):
    for y in range (len(flux[0])):
        y_ax = np.where(np.logical_and(wave > lower_b, wave < upper_b))[0]    
        y_ax = y_ax[2:28]
        
        flux_compare = [] 
        ivar_compare = []
        mask_compare = []
        result = []
        sumresult = []

        for i in y_ax:
            flux_compare.append(flux[x,y,i].flatten())
            ivar_compare.append(ivar[x,y,i].flatten())
            mask_compare.append(do_not_use[x,y,i].flatten())

        flux_compare = np.array(flux_compare)
        ivar_compare = np.array(ivar_compare)
        mask_compare = np.array(mask_compare)
        
        for i in range(len(y_ax)):
            sigma = np.sqrt(1 / ivar_compare[i]) if ivar_compare[i] != 0 else 0
        
            if flux_compare[i] < (3 * sigma):
                result.append(0)
            else:
                result.append(flux_compare[i])
            sumresult.append(np.sum(result))
            result = []
    
        index = np.where(y_ax == y_ax[-2])[0][0]

        a = [wave[y_ax[-2]], wave[y_ax[-1]]]
        b = [sumresult[index], sumresult[index + 1]]

        m = (b[1] - b[0])/(b[1] - b[0])

        c = b[0] - m*a[0]
        
        def func(b, m, c):
            a = (b - c)/m
            return a

        xval = func(sumresult[0], m, c)
        
        xcoord = []
        ycoord = []
        dlmd = []

        for i in range(y_ax[0], y_ax[-1]):
            xcoord.append(wave[i])
    
        xcoord.append(xval)
        ycoord = sumresult[:-1]
        ycoord.append(sumresult[0])
        
        for i in range(len(xcoord)-1):
            dlmd.append(xcoord[i+1] - xcoord[i])

        area = ycoord[0] * dlmd[0] + ycoord[-1] * dlmd[-1]

        for i in range(1, len(ycoord)-1):
            area += ycoord[i] * (dlmd[i-1] + dlmd[i])

        all_under = (1/2) * area

        rectangle = ycoord[0] * (xcoord[-1] - xcoord[0])

        flux_area_5007 = all_under - rectangle

        OIII_5007.append(flux_area_5007)
        
print (OIII_5007, len(OIII_5007))

<ipython-input-8-6e9fe5f9521a>:42: RuntimeWarning: invalid value encountered in float_scalars
  m = (b[1] - b[0])/(b[1] - b[0])
<ipython-input-8-6e9fe5f9521a>:42: RuntimeWarning: invalid value encountered in long_scalars
  m = (b[1] - b[0])/(b[1] - b[0])


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.5282461260760524, 0.5184078185901191, 0.45573102235377777, 0.44929607354277934, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.5252099087099362, 0.5155125079969453, 0.45377603757479706, 0.4474865463940601, 0.39025908483413513, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [9]:
OIII_5007 = np.array(OIII_5007)
OIII_5007[np.isnan(OIII_5007)] = 0
em = 0
for i in OIII_5007:
    em += i
    
em

5343.4527325494055